In [6]:
from bs4 import BeautifulSoup
import requests
import re 

In [ ]:
# This is the piece of code to scrape all of the urls - takes about 7mins without scraping

# full_list_url = []

# for i in range(1,333):
#     base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={i}&orderBy=relevance"
#     r = requests.get(base_url)
#     soup = BeautifulSoup(r.content, "html.parser")
    
#     for elem in soup.find_all("a", attrs={"class":"card__title-link"}):
#          full_list_url.append(elem.get('href'))
#          with open("full_list.txt", "a") as f:
#             f.write(elem.get('href'))
#             f.write('\n')
            
# print("Written to text file Successfully!")
# print(len(full_list_url))

In [3]:
# TEST THREADING
import requests
from bs4 import BeautifulSoup
import threading
import time

# Function to scrape URLs
def scrape_urls(page_num):
    base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={page_num}&orderBy=relevance"
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    urls = []
    for elem in soup.find_all("a", attrs={"class": "card__title-link"}):
        urls.append(elem.get('href'))
        
    # Save URLs to file - full_list.txt (local storage)
    with open("full_list.txt", "a") as f:
        for url in urls:
            f.write(url + '\n')
    return urls

def thread_scraping():
    full_list_url = []
    num_pages = 333

    # Create a list to store threads
    threads = []
    start_time = time.time()  # Start timer
    
    # Create and start threads
    for i in range(1, num_pages + 1):
        t = threading.Thread(target=lambda: full_list_url.extend(scrape_urls(i)))
        threads.append(t)
        t.start()

    # Wait for all threads to complete and then join
    for t in threads:
        t.join()

    end_time = time.time()  # Stop timer
    execution_time = end_time - start_time

    print("Scraping completed!")
    print("Total URLs scraped:", len(full_list_url))
    print("Total time:", execution_time, "seconds")


thread_scraping()

Scraping completed!
Total URLs scraped: 19980
Total time: 58.4198853969574 seconds


In [45]:
import requests
from bs4 import BeautifulSoup as soup
import re
import json

#https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263        

url = "https://www.immoweb.be/en/classified/apartment/for-sale/deinze/9800/10663064"
house_page = requests.get(url)
house_page = soup(house_page.text, 'html.parser')

regex = r"window.classified = (\{.*\})"

script = house_page.find('div',attrs={"id":"main-container"}).script.text
script = re.findall(regex, script)
 
script = json.loads(script[0])
#print(script['property']['location']['locality'])

{'id': 10663064,
 'cluster': None,
 'customers': [{'id': 2118760,
   'type': 'AGENCY',
   'email': '199809@secure.fortissimmo.net',
   'logoUrl': 'https://static.immoweb.be/logos/2118760.gif?cache=2016101403100',
   'phoneNumber': '+32 9 225 25 24',
   'mobileNumber': None,
   'name': 'Landbergh',
   'website': 'http://www.landbergh.be/',
   'location': {'country': 'Belgium',
    'region': None,
    'province': 'East Flanders',
    'district': 'Gent',
    'locality': 'Gentbrugge',
    'postalCode': '9050',
    'street': 'Brusselsesteenweg 690',
    'number': None,
    'box': None,
    'propertyName': None,
    'floor': None,
    'latitude': 51.03,
    'longitude': 3.77,
    'distance': None,
    'approximated': None,
    'regionCode': None,
    'type': None,
    'hasSeaView': None,
    'pointsOfInterest': None,
    'placeName': None},
   'ipiNo': '505.931',
   'isOwner': True,
   'contactHoursMobile': 'anytime',
   'contactHoursLandline': 'anytime',
   'salesRepresentative': None,
   '

In [53]:
def scrape_house(url):
    """Scrapes all the info from a house listing"""

    # Get the house listing and make a soup
    house_page = requests.get(url)
    house_page = soup(house_page.text, 'html.parser')
    final_dictionary = {}

    # Get the hidden info from the java script
    regex = r"window.classified = (\{.*\})"
    script = house_page.find('div',attrs={"id":"main-container"}).script.text
    script = re.findall(regex, script)
    script = json.loads(script[0])

    final_dictionary = {}
        #Locality
    final_dictionary['locality']= script['property']['location']['locality']
    #type of property
    final_dictionary['type of property']= script['property']['type']
    #subtype of property
    final_dictionary['subtype of property']= script['property']['subtype']
    #price
    final_dictionary['price']= script['price']['mainValue']
    #- Number of rooms
    final_dictionary['number_rooms'] = script['property']['bedroomCount']
    # living area
    final_dictionary['living_area'] = script['property']['netHabitableSurface']
    # Fully equipped kitchen (Yes/No)
    final_dictionary['kitchen'] = script['property']['kitchen']['type']
    # NOT INSTALLED / INSTALLED 
    # Furnished (Yes/No)
    final_dictionary['furnished'] = script['transaction']['sale']['isFurnished']
    # Open fire (Yes/No)
    final_dictionary['fireplace'] = script['property']['fireplaceCount']
    # Terrace (Yes/No)
    final_dictionary['terrace'] = script['property']['hasTerrace']
    # If yes: Area
    final_dictionary['terrace_area'] = script['property']['terraceSurface']
    # Garden
    final_dictionary['garden'] = script['property']['hasGarden']
    #- If yes: Area
    final_dictionary['garden_area'] = script['property']['gardenSurface']
    # Surface of the land
    final_dictionary['surface_land'] = script['property']['land']['surface']
    # Surface area of the plot of land - TO ASK
    # Number of facades
    final_dictionary['number_facades'] = script['property']['building']['facadeCount']
    # Swimming pool (Yes/No)
    final_dictionary['swimming_pool'] =  script['property']['hasSwimmingPool']
    # State of the building (New, to be renovated, ...)
    final_dictionary['building_state'] = script['property']['building']['condition']

    return final_dictionary




In [54]:
import requests
from bs4 import BeautifulSoup as soup
import re
import json
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor, as_completed
import time
import pandas as pd

houses_links = ["https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263", "https://www.immoweb.be/en/classified/house/for-sale/lede/9340/10660142", "https://www.immoweb.be/en/classified/house/for-sale/russeignies/7750/10660333", "https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263"]

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(scrape_house, url) for url in houses_links]
    results =  [item.result() for item in futures]
    df = pd.DataFrame(results)

In [55]:
df

,locality,type of property,subtype of property,price,number_rooms,living_area,kitchen,furnished,fireplace,terrace,terrace_area,garden,garden_area,surface_land,number_facades,swimming_pool,building_state
0,Libin,HOUSE,HOUSE,299900,3,124,NOT_INSTALLED,False,None,True,8.0,True,257,320,3,False,AS_NEW
1,Lede,HOUSE,HOUSE,219000,3,170,INSTALLED,None,None,True,NaN,True,450,564,3,False,TO_BE_DONE_UP
2,Russeignies,HOUSE,HOUSE,275000,5,172,INSTALLED,False,None,True,80.0,True,210,818,4,False,GOOD
3,Libin,HOUSE,HOUSE,299900,3,124,NOT_INSTALLED,False,None,True,8.0,True,257,320,3,False,AS_NEW
